In [0]:
from databricks.sdk.service.jobs import JobSettings as Job
from databricks.sdk import WorkspaceClient
from databricks.sdk.service import jobs, compute
import requests
import json
import os
import uuid
import os
from itertools import combinations

In [0]:
dbutils.widgets.text("jobId", "", "Job Id for UCD Copy using dbutils")

dbutils.widgets.dropdown("debug", "True", ["True", "False"], "Debug")
dbutils.widgets.dropdown("s3_batch_copy_id", "Test1", ["Test1"], "Batch Copy Id")


job_id = dbutils.widgets.get("jobId")
debug = dbutils.widgets.get("debug")
s3_batch_copy_id = dbutils.widgets.get("s3_batch_copy_id")

max_workers = 4 #no of cores

runJob=True

In [0]:

if s3_batch_copy_id == 'Test1':
    root_src_folder_1 = "s3://databricks-e2demofieldengwest/dom_rodrigues/folder_1/"
    root_dest_folder_1 = "s3://databricks-e2demofieldengwest/dom_rodrigues/folder_3/"

    root_src_folder_2 = "s3://databricks-e2demofieldengwest/dom_rodrigues/folder_3/"
    root_dest_folder_2 = "s3://databricks-e2demofieldengwest/dom_rodrigues/folder_4/"


    #Add more src and dest folder as needed and adjust the concurrency in the job
    src_dest_folder_tuple = [(root_src_folder_1, root_dest_folder_1)]                                               

In [0]:
def validate_folder_structure_for_copy(src_dest_folder_tuple):
    """
    Validates the folder structure for copy operation
    """

    df = pd.DataFrame(src_dest_folder_tuple, columns=['src', 'dst'])

    if  df['src'].duplicated().any():
        raise Exception("Duplicate src folders found. Please check the src and dest array and try again.")
    if  df['dst'].duplicated().any():
        raise Exception("Duplicate dst folders found. Please check the src and dest array and try again.")
                        
    def check_for_nested_folder_structure(pdf_series):

        def is_nested(child, parent):
            child = os.path.normpath(child)
            parent = os.path.normpath(parent)
            if child == parent:
                return False
            return os.path.commonpath([child, parent]) == parent
        
        folders = set(pdf_series)

        nested = [
            (child, parent)
            for child, parent in combinations(folders, 2)
            if is_nested(child, parent)
        ]
        nested_df = pd.DataFrame(nested, columns=['Nested Folder', 'Parent Folder'])
        return nested_df

    src_nested_df = check_for_nested_folder_structure(df['src'])

    dest_nested_df = check_for_nested_folder_structure(df['src'])

    if src_nested_df.empty  is False or dest_nested_df.empty is False :
        print(f"❌ Nested src or dest folders found. Please check the src and dest array and try again.")
        print(f"\n Src: {src_nested_df}\n Dest: {dest_nested_df}")
        raise Exception(f"Nested  src or dest folders found. Please check the dest and dest array and try again.")

validate_folder_structure_for_copy(src_dest_folder_tuple)      

In [0]:
if runJob:
    for root_src_folder, root_dest_folder in src_dest_folder_tuple:
        databricks_instance = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().getOrElse(None)

        access_token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

        headers = {
            "Authorization": f"Bearer {access_token}",
            "Content-Type": "application/json"
        }

        s3_cp_task_parameters= {
        "root_src_folder": root_src_folder,
        "root_dest_folder": root_dest_folder,
        "max_workers": max_workers
        }
        
        print(f"Running Copy job from {root_src_folder} to {root_dest_folder}")
        payload = {
            "job_id": job_id,
            "notebook_params": s3_cp_task_parameters,
            "idempotency_token": str(uuid.uuid4())
        }

        response = requests.post(
            f"{databricks_instance}/api/2.0/jobs/run-now",
            headers=headers,
            json=payload
        ).json()

        print(response)

Copying from s3://databricks-e2demofieldengwest/dom_rodrigues/folder_1/ to s3://databricks-e2demofieldengwest/dom_rodrigues/folder_3/
{'run_id': 357001791051102, 'number_in_job': 357001791051102}
